使用モジュール

In [2]:
from time import sleep
import pandas as pd
import requests  # クローリングモジュール
from bs4 import BeautifulSoup  # スクレイピングモジュール

変数

In [3]:
base_url = "https://employment.en-japan.com"
url = "https://employment.en-japan.com/wish/search_list/?occupation=351018_401015_401020&areaid=11_23&companytype=0&worktype=0&sort=wish&pagenum={}"
max_page_index = 13
data_list = []


In [4]:
for i in range(max_page_index):
    print('d_list:', len(data_list))
    access_url = url.format(i+1)
    sleep(2)

    res = requests.get(access_url)
    res.raise_for_status()          #エラーで終了させる
    html = res.text
    soup = BeautifulSoup(html, "html.parser")
    jobs = soup.find_all('div', class_='jobNameArea')
    for job in jobs:
        company_name = job.find('span', class_='company').text
        page_url = base_url + job.find('a').get('href')
        if 'fromSearch' in page_url:
            path = page_url.replace(base_url+'/desc_eng_', '')
            e_index = path.rfind('/')
            company_id = path[:e_index]
            page_url = f'https://en-gage.net/recruit/?getFromEmploy={company_id}'
        sleep(3)
        page_r = requests.get(page_url)
        page_r.raise_for_status()
        page_soup = BeautifulSoup(page_r.content, 'lxml')
        company_url = None
        if 'PK' in page_url:
            company_info_block = [h2 for h2 in page_soup.find_all(
                'h2', class_='text') if '会社概要' in h2.text][0]
            company_summary = company_info_block.parent.parent
            for table_row in company_summary.find_all('tr'):
                if table_row.find('th').text == '企業ホームページ':
                    company_url = table_row.find('td').find('a').text
        elif 'getFromEmploy' in page_url:
            company_summary = page_soup.find('table', class_='companyTable')
            for table_row in company_summary.find_all('tr'):
                if table_row.find('th').text == '企業WEBサイト':
                    company_url = table_row.find('td').find('a').get('href')
        else:
            raise
        data_list.append({
            'company_name': company_name,
            'company_url': company_url
        })
        print(data_list[-1])


df = pd.DataFrame(data_list)
df.to_csv('enjapan_company_list.csv', index=False, encoding='utf-8-sig')


d_list: 0
{'company_name': '株式会社セラク（東証プライム市場上場企業）', 'company_url': 'http://www.seraku.co.jp/'}
{'company_name': 'ラ・ハイナ株式会社', 'company_url': 'https://la-haina.jp/'}
{'company_name': '株式会社エーエスエル', 'company_url': 'https://www.asl.co.jp/'}
{'company_name': '株式会社ダイヤコーポレーション', 'company_url': 'https://daiya-corp.com'}
{'company_name': '株式会社ワールドインテック SI事業部（ワールドホールディングスグループ）', 'company_url': 'https://www.witc.co.jp/'}
{'company_name': '株式会社レインオンファニー', 'company_url': 'https://www.reinonfunny.co.jp/'}
{'company_name': '株式会社ビーネックステクノロジーズ', 'company_url': 'https://www.benextgroup.net/bnt/career_engineer'}
{'company_name': '株式会社リクルートＲ＆Ｄスタッフィング', 'company_url': 'https://www.recruit-rds.co.jp/'}
{'company_name': '株式会社ECS', 'company_url': 'https://ecs.jp/'}
{'company_name': '株式会社レインオンファニー', 'company_url': 'https://www.reinonfunny.co.jp/'}
{'company_name': '株式会社UNITE', 'company_url': 'https://www.u-nite.co.jp/'}
{'company_name': '株式会社ウィズテック', 'company_url': 'https://wiz-tech.jp/'}
{'company_name': '株式会社